In [11]:
pip install PyPortfolioOpt pandas_datareader plotly absl-py


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [12]:
import numpy as np
import pandas as pd
# setting the seed allows for reproducible results
import backtrader as bt
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import CLA
from pypfopt import black_litterman
from pypfopt import BlackLittermanModel
from pypfopt import HRPOpt
from pypfopt import EfficientSemivariance
from pandas_datareader import data as pdr
from datetime import datetime
from scipy.special import softmax
import yfinance as yf

#--------------

#-------------

import plotly.graph_objects as go

float_formatter = "{:.4f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

np.random.seed(123)
%matplotlib inline

very_small_float = 1.0#0.000000001

In [13]:
from bt_testcode import backtest, DynRebalance, get_name, plotAllocation, getStatistic, calc_portfolio_values
from bt_testcode import plotPortfolioReturns, get_portfolio_values

# Model definition

In [14]:
from scipy import optimize 

def MaximizeSharpeRatioOptmzn(MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize):
    
    # define maximization of Sharpe Ratio using principle of duality
    def  f(x, MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize):
        funcDenomr = np.sqrt(np.matmul(np.matmul(x, CovarReturns), x.T) )
        funcNumer = np.matmul(np.array(MeanReturns),x.T)-RiskFreeRate
        func = -(funcNumer / funcDenomr)
        #print(func)
        return func

    #define equality constraint representing fully invested portfolio
    def constraintEq(x):
        A=np.ones(x.shape)
        b=1
        constraintVal = np.matmul(A,x.T)-b 
        return constraintVal
    
    
    #define bounds and other parameters
    xinit=np.repeat(0.33, PortfolioSize)
    cons = ({'type': 'eq', 'fun':constraintEq})
    lb = 0
    ub = 1
    bnds = tuple([(lb,ub) for x in xinit])
    
    #print('xinit',xinit.shape)
    #invoke minimize solver
    opt = optimize.minimize (f, x0 = xinit, args = (MeanReturns, CovarReturns,\
                             RiskFreeRate, PortfolioSize), method = 'SLSQP',  \
                             bounds = bnds, constraints = cons, tol = 10**-3)
    
    return opt

def get_weight(data, meanReturns, covReturns):
    
    #set portfolio size
    portfolioSize = data.shape[1]

    #set risk free asset rate of return
    Rf=0  #  average risk  free rate of return in USA 
    annRiskFreeRate = Rf/100

    #compute daily risk free rate in percentage
    r0 = (np.power((1 + annRiskFreeRate),  (1.0 / 360.0)) - 1.0) * 100 
    #print('\nRisk free rate (daily %): ', end="")
    #print ("{0:.3f}".format(r0)) 

    #initialization
    xOptimal =[]
    minRiskPoint = []
    expPortfolioReturnPoint =[]
    maxSharpeRatio = 0

    #compute maximal Sharpe Ratio and optimal weights
    result = MaximizeSharpeRatioOptmzn(meanReturns, covReturns, r0, portfolioSize)
    xOptimal.append(result.x)

        
    #compute risk returns and max Sharpe Ratio of the optimal portfolio   
    xOptimalArray = np.array(xOptimal)
    Risk = np.matmul((np.matmul(xOptimalArray,covReturns)), np.transpose(xOptimalArray))
    expReturn = np.matmul(np.array(meanReturns),xOptimalArray.T)
    annRisk =   np.sqrt(Risk*252) 
    annRet = 252*np.array(expReturn) 
    maxSharpeRatio = (annRet-Rf)/annRisk 

    #set precision for printing results
    np.set_printoptions(precision=3, suppress = True)

    return (xOptimalArray.T).flatten()

In [15]:
class Model:
    def __init__(self):
        self.data = None
        self.isFirst = True
    
    def getModel():
        return None
    
    def resetModel():
        self.data = None
        self.isFirst = True
        pass

    # a nonconvex objective from  Kolm et al (2014)
    def deviation_risk_parity(self, w, cov_matrix):
        diff = w * np.dot(cov_matrix, w) - (w * np.dot(cov_matrix, w)).reshape(-1, 1)
        return (diff ** 2).sum().sum()


    #def get_allocations(self, data, model_n = 'MeanVariance', isEmsRet = False, isExpCov = False, isCleanWeight = False, upd_period = 0):
    def get_allocations(self, data, **param):

        data = pd.DataFrame(data)

        model_n = param['model_n']
        isEmsRet = param['isEmsRet']
        isExpCov = param['isExpCov']
        isCleanWeight = param['isCleanWeight']
        upd_period = param['upd_period']
        isDataCollecting = param['collectData']

        


        #data = data.fillna(0.0)
        if(self.isFirst == True) or (upd_period == 0) or (isDataCollecting==False):
            self.isFirst = False
            self.data = data.copy()
        else:
            self.data = self.data.append(data.iloc[-upd_period:])

       # print(data.iloc[-upd_period:])

        mu = expected_returns.mean_historical_return(self.data).fillna(0.0).values
        if isEmsRet == True:
            mu = expected_returns.ema_historical_return(self.data).fillna(0.0).values
    
        S = risk_models.sample_cov(self.data).fillna(0.0).values
        if isExpCov == True:    
            S = risk_models.exp_cov(self.data).fillna(0.0).values
        
        if model_n == 'CLA':
            ef = CLA(mu, S)
            weights = ef.max_sharpe()
            cleaned_weights = ef.clean_weights()
            #hrp.portfolio_performance(verbose=True)
            if isCleanWeight == True:
                return pd.DataFrame(cleaned_weights, index=[0]).to_numpy()[0]
            return pd.DataFrame(weights, index=[0]).to_numpy()[0]

        elif model_n == 'HRP_':
            hrp = HRPOpt(self.data.pct_change())
            weights = hrp.optimize()
            cleaned_weights = hrp.clean_weights()        
            if isCleanWeight == True:
                return pd.DataFrame(cleaned_weights, index=[0]).to_numpy()[0]
            return pd.DataFrame(weights, index=[0]).to_numpy()[0]

        elif model_n == 'HRP':
            hrp = HRPOpt(self.data.pct_change(),S)
            weights = hrp.optimize()
            cleaned_weights = hrp.clean_weights()        
            if isCleanWeight == True:
                return pd.DataFrame(cleaned_weights, index=[0]).to_numpy()[0]
            return pd.DataFrame(weights, index=[0]).to_numpy()[0]


        elif model_n == 'nonconvex':
            ef = EfficientFrontier(mu, S)
            weights = ef.nonconvex_objective(self.deviation_risk_parity, ef.cov_matrix)
            cleaned_weights = ef.clean_weights()        
            if isCleanWeight == True:
                return pd.DataFrame(cleaned_weights, index=[0]).to_numpy()[0]
            return pd.DataFrame(weights, index=[0]).to_numpy()[0]

        elif model_n == 'random':
            a = np.random.uniform(size=[self.data.shape[1]])
            return softmax(a)
        elif model_n == 'equal_weight':
            a = np.array([1/self.data.shape[1]]*self.data.shape[1])
            return a
        else:
           # print('DEFAULT MODEL!')
            return get_weight(self.data, mu, S)

        return []

In [16]:

ticker = ['MSFT', 'AMD', 'C', 'BA', 'AMZN', 'AAPL','MO', 'AEE', 'AIG', 'WY']
history_range = {'start': datetime(2018, 1, 1), 
                 'end': datetime(2020, 10, 1)}
stock = pdr.DataReader(ticker, 
                start=history_range['start'],
                end=history_range['end'],
                data_source='yahoo')
table = yf.download(ticker, start=history_range['start'],
                    end=history_range['end'])['Adj Close']
table.head()


[*********************100%***********************]  10 of 10 completed


,AAPL,AEE,AIG,AMD,AMZN,BA,C,MO,MSFT,WY
Date,,,,,,,,,,
2018-01-02,41.018421,51.728882,52.261723,10.98,59.450500,282.886383,64.162941,51.659672,81.168503,29.101162
2018-01-03,41.011288,51.463116,52.841434,11.55,60.209999,283.801270,64.361412,51.469810,81.546227,29.356726
2018-01-04,41.201778,50.878410,53.052235,12.12,60.479500,282.724396,65.155258,51.265324,82.263962,29.117657
2018-01-05,41.670872,50.842960,53.271820,11.88,61.457001,294.322327,65.068954,51.411388,83.283882,29.076426
2018-01-08,41.516102,51.445393,52.920475,12.28,62.343498,295.570709,64.309631,51.345650,83.368889,29.266043


In [17]:
stock_close = stock.iloc[:125]['Close'].reset_index(drop=True)
stock_close.head()

Symbols,MSFT,AMD,C,BA,AMZN,AAPL,MO,AEE,AIG,WY
0,85.949997,10.98,74.360001,296.839996,59.450500,43.064999,70.739998,58.389999,59.500000,35.299999
1,86.349998,11.55,74.589996,297.799988,60.209999,43.057499,70.480003,58.090000,60.160000,35.610001
2,87.110001,12.12,75.510002,296.670013,60.479500,43.257500,70.199997,57.430000,60.400002,35.320000
3,88.190002,11.88,75.410004,308.839996,61.457001,43.750000,70.400002,57.389999,60.650002,35.270000
4,88.279999,12.28,74.529999,310.149994,62.343498,43.587502,70.309998,58.070000,60.250000,35.500000


In [18]:
param={}
param ={'model_params':
    {'model_n':'equal_weight',
    'collectData':False,
    'isEmsRet':False,
    'isExpCov':False,
    'isCleanWeight':False,
    'upd_period': 0}}

In [19]:
m = Model()
m.get_allocations(stock_close, **param['model_params'])

array([0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
       0.1000, 0.1000])

In [20]:

n = [0, 200, 250, 300, 350, 400, 500, 700, 1000, 1300]
def getReturnFromAlloc(n, model_n):
    r_list = []
    param={}
    param ={'model_params':
        {'model_n':model_n,
        'collectData':False,
        'isEmsRet':True,
        'isExpCov':True,
        'isCleanWeight':False,
        'upd_period': 0}}
    #print('ret, std, sharp')
    for i in n:
        stock_close = stock.iloc[i:i+125]['Close'].reset_index(drop=True).fillna(very_small_float)
        m = Model()
        alloc = m.get_allocations(stock_close.values, **param['model_params'])
        r_list.append((get_portfolio_values(alloc, stock_close)))
        #print(r_list[-1])
    return r_list

rl = getReturnFromAlloc(n, 'CLA')
for r, s, sp in rl:
    print("{:.4f}  {:.4f}  {:.4f}".format(r,s,sp))

IndexError: single positional indexer is out-of-bounds

In [ ]:
model_List = ['MeanVariance', 'CLA', 'HRP', 'nonconvex', 'random', 'equal_weight']
res = {}
for mn in model_List:
    rl = getReturnFromAlloc(n, mn)
    sharp = []
    for r,s,sp in rl:
        sharp.append(sp)
    res[mn] = sharp
pd.DataFrame(res).to_csv('models_alloc_results.csv')
pd.DataFrame(res)

In [ ]:
ticker = ['SPY', 'TLT', 'GLD', 'RWO']
history_range = {'start': datetime(2010, 1, 1), 
                 'end': datetime(2020, 10, 1)}

stock = pdr.DataReader(ticker, 
                start=history_range['start'],
                end=history_range['end'],
                data_source='yahoo')

res = {}
for mn in model_List:
    rl = getReturnFromAlloc(n, mn)
    sharp = []
    for r,s,sp in rl:
        sharp.append(sp)
    res[mn] = sharp
pd.DataFrame(res).to_csv('models_alloc_results_2.csv')
pd.DataFrame(res)

## Test

In [ ]:
from bt_testcode import backtest, DynRebalance, get_name, plotAllocation, getStatistic, plotPorfolioReturns

In [ ]:
def executeTest(d_stat, d_res, params, config_cerebro):

    def setParams(params, model_n, collectData):
        params['model'] = Model()
        params['model_params']['model_n'] = model_n
        params['model_params']['collectData'] = collectData
        return params


    ah = {}
    print('MeanVariance')
    params = setParams(params, model_n='MeanVariance', collectData=False)
    d_stat[get_name(params)], d_res[get_name(params)], ah[get_name(params)] = backtest(config_cerebro, DynRebalance, m_name=get_name(params), plot=False, **params)

    print('MeanVariance + collectData = True')
    #params['collectData'] = True
    params = setParams(params, model_n='MeanVariance', collectData=True)
    d_stat[get_name(params)], d_res[get_name(params)], ah[get_name(params)] = backtest(config_cerebro, DynRebalance, m_name=get_name(params), plot=False, **params)

    print('CLA')
    #params['collectData'] = False
    #params['model_n'] =  'CLA'
    params = setParams(params, model_n='CLA', collectData=False)
    d_stat[get_name(params)], d_res[get_name(params)], ah[get_name(params)] = backtest(config_cerebro, DynRebalance, m_name=get_name(params), plot=False, **params)

    print('CLA + collectData = True')
    #params['collectData'] = True
    #params['model_n'] =  'CLA'
    params = setParams(params, model_n='CLA', collectData=True)
    d_stat[get_name(params)], d_res[get_name(params)], ah[get_name(params)] = backtest(config_cerebro, DynRebalance, m_name=get_name(params), plot=False, **params)

    print('HRP')
    #params['collectData'] = False
    #params['model_n'] =  'HRP'
    params = setParams(params, model_n='HRP', collectData=False)
    d_stat[get_name(params)], d_res[get_name(params)], ah[get_name(params)] = backtest(config_cerebro, DynRebalance, m_name=get_name(params), plot=False, **params)

    print('HRP + collectData = True')
   # params['collectData'] = True
   # params['model_n'] =  'HRP'
    params = setParams(params, model_n='HRP', collectData=True)
    d_stat[get_name(params)], d_res[get_name(params)], ah[get_name(params)] = backtest(config_cerebro, DynRebalance, m_name=get_name(params), plot=False, **params)

    print('noncovex')
    #params['collectData'] = False
    #params['model_n'] =  'nonconvex'
    params = setParams(params, model_n='nonconvex', collectData=False)
    d_stat[get_name(params)], d_res[get_name(params)], ah[get_name(params)] = backtest(config_cerebro, DynRebalance, m_name=get_name(params), plot=False, **params)

    print('nonconvex + collectData = True')
#    params['collectData'] = True
#    params['model_n'] =  'nonconvex'
    params = setParams(params, model_n='nonconvex', collectData=True)
    d_stat[get_name(params)], d_res[get_name(params)], ah[get_name(params)] = backtest(config_cerebro, DynRebalance, m_name=get_name(params), plot=False, **params)

    print('random')
    #params['collectData'] = False
    #params['model_n'] =  'random'
    params = setParams(params, model_n='random', collectData=False)
    d_stat[get_name(params)], d_res[get_name(params)], ah[get_name(params)] = backtest(config_cerebro, DynRebalance, m_name=get_name(params), plot=False, **params)

    print('equal_weight')
    params = setParams(params, model_n='equal_weight', collectData=False)
    d_stat[get_name(params)], d_res[get_name(params)], ah[get_name(params)] = backtest(config_cerebro, DynRebalance, m_name=get_name(params), plot=False, **params)

    print('Benchmark')
    config_cerebro['stocks_df'] = None
    config_cerebro['benchmark_df'] = None
    config_cerebro['assets'] = [config_cerebro['benchmark']]
    params = setParams(params, model_n='CLA', collectData=False)
    d_stat['Benchmark'], d_res['Benchmark'], ah[get_name(params)] = backtest(config_cerebro, DynRebalance, m_name='Benchmark', plot=False, **params)
    
    return d_stat, d_res, ah

# Test 2008-2010

In [ ]:
config_cerebro = {
    'assets':['MSFT', 'AMD', 'C', 'BA', 'AMZN', 'AAPL','MO', 'AEE', 'AIG', 'WY'],
    'benchmark':'SPY',
    'startd': datetime(2008, 1, 1),
    'endd'  : datetime(2010, 12, 31),
    'set_coc' : True,
    'set_coo' : False,
    'cash': 11000.0,
    'stocks_df':None,
    'benchmark_df':None
}

params = {
    'DataCounter': 125,
    'RebalanceDay': 22,
    'reserveCash': 1000.0,
    'printlog': 0,
    'benchmarkUse': True,
    'resetModel':False, 
    'resetModelStep':4,
    'model':Model(),
    'model_params':{
        'model_n':'CLA',
        'collectData':False,
        'isEmsRet':True,
        'isExpCov':True,
        'isCleanWeight':False,
        'upd_period': 22}
}
d_res = {}
d_stat = {}

In [ ]:
config_cerebro['stocks_df'] = pdr.DataReader(config_cerebro['assets'], 
                start=config_cerebro['startd'],
                end=config_cerebro['endd'],
                data_source='yahoo')
config_cerebro['benchmark_df'] = pdr.DataReader(config_cerebro['benchmark'], 
                start=config_cerebro['startd'],
                end=config_cerebro['endd'],
                data_source='yahoo')

## Random weight

In [ ]:
params['model_params']['model_n'] =  'random'

d_stat[get_name(params)], d_res[get_name(params)], ah = backtest(config_cerebro, DynRebalance, m_name=get_name(params), plot=False, **params)
display(d_stat[get_name(params)])
d_res[get_name(params)].plot()

In [ ]:
plotAllocation(ah.dropna())

## MeanVariance

No Data collecting

In [ ]:
params['model'] = Model()
params['model_params']['model_n'] = 'MeanVariance'


d_stat[get_name(params)], d_res[get_name(params)], ah = backtest(config_cerebro, DynRebalance, m_name=get_name(params), plot=False, **params)
display(d_stat[get_name(params)])
d_res[get_name(params)].plot()

In [ ]:
plotAllocation(ah.dropna())

with data collecting

In [ ]:
params['model'] = Model()
params['model_params']['model_n'] = 'MeanVariance'
params['model_params']['collectData'] = True

d_stat[get_name(params)], d_res[get_name(params)], ah = backtest(config_cerebro, DynRebalance, m_name=get_name(params), plot=False, **params)
display(d_stat[get_name(params)])
d_res[get_name(params)].plot()

In [ ]:
plotAllocation(ah.dropna())

## CLA

In [ ]:
params['model'] = Model()
params['model_params']['model_n'] = 'CLA'
params['model_params']['collectData'] = False

d_stat[get_name(params)], d_res[get_name(params)], ah = backtest(config_cerebro, DynRebalance, m_name=get_name(params), plot=False, **params)
display(d_stat[get_name(params)])
d_res[get_name(params)].plot()

In [ ]:
plotAllocation(ah.dropna())

In [ ]:
params['model'] = Model()
params['model_params']['model_n'] = 'CLA'
params['model_params']['collectData'] = True
d_stat[get_name(params)], d_res[get_name(params)], ah = backtest(config_cerebro, DynRebalance, m_name=get_name(params), plot=False, **params)
display(d_stat[get_name(params)])
d_res[get_name(params)].plot()

In [ ]:
plotAllocation(ah.dropna())

## HRP

In [ ]:
params['model'] = Model()
params['model_params']['model_n'] = 'HRP'
params['model_params']['collectData'] = False

d_stat[get_name(params)], d_res[get_name(params)], ah = backtest(config_cerebro, DynRebalance, m_name=get_name(params), plot=False, **params)
display(d_stat[get_name(params)])
d_res[get_name(params)].plot()

In [ ]:
plotAllocation(ah.dropna())

In [ ]:
params['model'] = Model()
params['model_params']['model_n'] = 'HRP'
params['model_params']['collectData'] = True

d_stat[get_name(params)], d_res[get_name(params)], ah = backtest(config_cerebro, DynRebalance, m_name=get_name(params), plot=False, **params)
display(d_stat[get_name(params)])
d_res[get_name(params)].plot()

In [ ]:
plotAllocation(ah.dropna())

## nonconvex

In [ ]:
params['model'] = Model()
params['model_params']['model_n'] = 'nonconvex'
params['model_params']['collectData'] = False

d_stat[get_name(params)], d_res[get_name(params)], ah = backtest(config_cerebro, DynRebalance, m_name=get_name(params), plot=False, **params)
display(d_stat[get_name(params)])
d_res[get_name(params)].plot()

In [ ]:
plotAllocation(ah.dropna())

In [ ]:
params['model'] = Model()
params['model_params']['model_n'] = 'nonconvex'
params['model_params']['collectData'] = True

d_stat[get_name(params)], d_res[get_name(params)], ah = backtest(config_cerebro, DynRebalance, m_name=get_name(params), plot=False, **params)
display(d_stat[get_name(params)])
d_res[get_name(params)].plot()

In [ ]:
plotAllocation(ah.dropna())

In [ ]:
params['model'] = Model()
params['model_params']['model_n'] = 'equal_weight'
params['model_params']['collectData'] = False

d_stat[get_name(params)], d_res[get_name(params)], ah = backtest(config_cerebro, DynRebalance, m_name=get_name(params), plot=False, **params)
display(d_stat[get_name(params)])
d_res[get_name(params)].plot()

In [ ]:
params['model'] = Model()
config_cerebro['config_cerebro'] = ['SPY']
d_stat['Benchmark'], d_res['Benchmark'], ah = backtest(config_cerebro, DynRebalance, m_name='Benchmark', plot=False, **params)

In [ ]:

stat = getStatistic(d_stat)
stat.to_csv("models_stat_2008_2010.csv")
stat.sort_values(by=['Sharp_Ratio'], ascending=False)

In [ ]:
df = plotPorfolioReturns(d_res, None)
df.to_csv('models_drawdown_2008_2010.csv')

# TEST 2011-2017

In [ ]:
config_cerebro = {
    'assets':['MSFT', 'AMD', 'C', 'BA', 'AMZN', 'AAPL','MO', 'AEE', 'AIG', 'WY'],
    'benchmark':'SPY',
    'startd': datetime(2011, 1, 1),
    'endd'  : datetime(2017, 12, 31),
    'set_coc' : True,
    'set_coo' : False,
    'cash': 11000.0,
    'stocks_df':None,
    'benchmark_df':None
}

params = {
    'DataCounter': 125,
    'RebalanceDay': 22,
    'reserveCash': 1000.0,
    'printlog': 0,
    'benchmarkUse': True,
    'resetModel':False, 
    'resetModelStep':4,
    'model':Model(),
    'model_params':{
        'model_n':'CLA',
        'collectData':False,
        'isEmsRet':True,
        'isExpCov':True,
        'isCleanWeight':False,
        'upd_period': 22}
}
d_res = {}
d_stat = {}
config_cerebro['stocks_df'] = pdr.DataReader(config_cerebro['assets'], 
                start=config_cerebro['startd'],
                end=config_cerebro['endd'],
                data_source='yahoo')
config_cerebro['benchmark_df'] = pdr.DataReader(config_cerebro['benchmark'], 
                start=config_cerebro['startd'],
                end=config_cerebro['endd'],
                data_source='yahoo')

In [ ]:
d_stat, d_res, d_ah = executeTest(d_stat, d_res, params, config_cerebro)

In [ ]:
stat = getStatistic(d_stat)
stat.to_csv("models_stat_2011_2017.csv")
stat.sort_values(by=['Sharp_Ratio'], ascending=False)

In [ ]:
df = plotPorfolioReturns(d_res, None)
df.to_csv('models_drawdown_2011_2017.csv')

# TEST 2018-2021

In [ ]:
config_cerebro = {
    'assets':['MSFT', 'AMD', 'C', 'BA', 'AMZN', 'AAPL','MO', 'AEE', 'AIG', 'WY'],
    'benchmark':'SPY',
    'startd': datetime(2018, 1, 1),
    'endd'  : datetime(2021, 12, 31),
    'set_coc' : True,
    'set_coo' : False,
    'cash': 11000.0,
    'stocks_df':None,
    'benchmark_df':None
}

params = {
    'DataCounter': 125,
    'RebalanceDay': 22,
    'reserveCash': 1000.0,
    'printlog': 0,
    'benchmarkUse': True,
    'resetModel':False, 
    'resetModelStep':4,
    'model':Model(),
    'model_params':{
        'model_n':'CLA',
        'collectData':False,
        'isEmsRet':True,
        'isExpCov':True,
        'isCleanWeight':False,
        'upd_period': 22}
}
d_res = {}
d_stat = {}
config_cerebro['stocks_df'] = pdr.DataReader(config_cerebro['assets'], 
                start=config_cerebro['startd'],
                end=config_cerebro['endd'],
                data_source='yahoo')
config_cerebro['benchmark_df'] = pdr.DataReader(config_cerebro['benchmark'], 
                start=config_cerebro['startd'],
                end=config_cerebro['endd'],
                data_source='yahoo')

In [ ]:
d_stat, d_res, d_ah = executeTest(d_stat, d_res, params, config_cerebro)

In [ ]:
stat = getStatistic(d_stat)
stat.to_csv("models_stat_2018_2021.csv")
stat.sort_values(by=['Sharp_Ratio'], ascending=False)

In [ ]:
df = plotPorfolioReturns(d_res, None)
df.to_csv('models_drawdown_2018_2021.csv')

# TEST 2012-2016

In [ ]:
config_cerebro = {
    'assets':['MSFT', 'AMD', 'C', 'BA', 'AMZN', 'AAPL','MO', 'AEE', 'AIG', 'WY'],
    'benchmark':'SPY',
    'startd': datetime(2012, 1, 1),
    'endd'  : datetime(2016, 12, 31),
    'set_coc' : True,
    'set_coo' : False,
    'cash': 11000.0,
    'stocks_df':None,
    'benchmark_df':None
}

params = {
    'DataCounter': 125,
    'RebalanceDay': 22,
    'reserveCash': 1000.0,
    'printlog': 0,
    'benchmarkUse': True,
    'resetModel':False, 
    'resetModelStep':4,
    'model':Model(),
    'model_params':{
        'model_n':'CLA',
        'collectData':False,
        'isEmsRet':True,
        'isExpCov':True,
        'isCleanWeight':False,
        'upd_period': 22}
}
d_res = {}
d_stat = {}
config_cerebro['stocks_df'] = pdr.DataReader(config_cerebro['assets'], 
                start=config_cerebro['startd'],
                end=config_cerebro['endd'],
                data_source='yahoo')
config_cerebro['benchmark_df'] = pdr.DataReader(config_cerebro['benchmark'], 
                start=config_cerebro['startd'],
                end=config_cerebro['endd'],
                data_source='yahoo')

In [ ]:
d_stat, d_res, d_ah = executeTest(d_stat, d_res, params, config_cerebro)

In [ ]:
stat = getStatistic(d_stat)
stat.to_csv("models_stat_2012_2016.csv")
stat.sort_values(by=['Sharp_Ratio'], ascending=False)

In [ ]:
df = plotPorfolioReturns(d_res, None)
df.to_csv('models_drawdown_2012_2016.csv')

# 2016-2021 
SPY, TLT, GLD, IYR

In [ ]:
config_cerebro = {
    'assets':['SPY', 'TLT', 'GLD', 'RWO'],
    'benchmark':'SPY',
    'startd': datetime(2016, 1, 1),
    'endd'  : datetime(2021, 12, 31),
    'set_coc' : True,
    'set_coo' : False,
    'cash': 11000.0,
    'stocks_df':None,
    'benchmark_df':None
}

params = {
    'DataCounter': 125,
    'RebalanceDay': 22,
    'reserveCash': 1000.0,
    'printlog': 0,
    'benchmarkUse': True,
    'resetModel':False, 
    'resetModelStep':4,
    'model':Model(),
    'model_params':{
        'model_n':'CLA',
        'collectData':False,
        'isEmsRet':True,
        'isExpCov':True,
        'isCleanWeight':False,
        'upd_period': 22}
}
d_res = {}
d_stat = {}
config_cerebro['stocks_df'] = pdr.DataReader(config_cerebro['assets'], 
                start=config_cerebro['startd'],
                end=config_cerebro['endd'],
                data_source='yahoo')
config_cerebro['benchmark_df'] = pdr.DataReader(config_cerebro['benchmark'], 
                start=config_cerebro['startd'],
                end=config_cerebro['endd'],
                data_source='yahoo')

In [ ]:
d_stat, d_res, d_ah = executeTest(d_stat, d_res, params, config_cerebro)

In [ ]:
stat = getStatistic(d_stat)
stat.to_csv("models_stat_2016_2021_etf.csv")
stat.sort_values(by=['Sharp_Ratio'], ascending=False)

In [ ]:
df = plotPorfolioReturns(d_res, None)
df.to_csv('models_drawdown_2016_2021_etf.csv')

In [ ]:
ticker = ['SPY', 'TLT']#, 'GLD', 'RWO']
history_range = {'start': datetime(2007, 1, 1), 
                 'end': datetime(2020, 10, 1)}

stock = pdr.DataReader(ticker, 
                start=history_range['start'],
                end=history_range['end'],
                data_source='yahoo')
stock.head()

In [ ]:
config_cerebro = {
    'assets':['SPY', 'TLT'],#, 'GLD', 'RWO'],
    'benchmark':'SPY',
    'startd': datetime(2007, 1, 1),
    'endd'  : datetime(2021, 12, 31),
    'set_coc' : True,
    'set_coo' : False,
    'cash': 11000.0,
    'stocks_df':None,
    'benchmark_df':None
}

params = {
    'DataCounter': 125,
    'RebalanceDay': 22,
    'reserveCash': 1000.0,
    'printlog': 0,
    'benchmarkUse': True,
    'resetModel':False, 
    'resetModelStep':4,
    'model':Model(),
    'model_params':{
        'model_n':'CLA',
        'collectData':False,
        'isEmsRet':True,
        'isExpCov':True,
        'isCleanWeight':False,
        'upd_period': 22}
}
d_res = {}
d_stat = {}
config_cerebro['stocks_df'] = pdr.DataReader(config_cerebro['assets'], 
                start=config_cerebro['startd'],
                end=config_cerebro['endd'],
                data_source='yahoo')
config_cerebro['benchmark_df'] = pdr.DataReader(config_cerebro['benchmark'], 
                start=config_cerebro['startd'],
                end=config_cerebro['endd'],
                data_source='yahoo')

In [ ]:
d_stat, d_res, d_ah = executeTest(d_stat, d_res, params, config_cerebro)

In [ ]:
stat = getStatistic(d_stat)
stat.sort_values(by=['Sharp_Ratio'], ascending=False)